# P1: Single-Agent Campus Tour - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**P1: Single-Agent Campus Tour** - Planning visit waypoints within time windows under spatial constraints.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for P1: Single-Agent Campus Tour


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Location Setup Agent ---- #
    LS_Agent = Agent(
        name="Location Setup Agent",
        backstory="You define campus locations, distances between locations, and time windows for each location visit.",
        task_description="Set up campus map with locations (library, cafeteria, gym, student_center), distances between them, and time windows for visits.",
        task_expected_output="Structured campus map data: 1) Locations: library, cafeteria, gym, student_center, entrance. 2) Distances between locations (in minutes). 3) Time windows for each location: library [9:00-12:00], cafeteria [11:00-14:00], gym [15:00-18:00]."
    )
    
    # ---- Route Planning Agent ---- #
    RP_Agent = Agent(
        name="Route Planning Agent",
        backstory="You plan the optimal route to visit all required locations within their time windows while minimizing total travel time.",
        task_description="Create an optimal tour route starting and ending at entrance, visiting all locations (library, cafeteria, gym, student_center) within their time windows, minimizing total travel time.",
        task_expected_output="Optimized tour route with visit sequence, arrival times at each location, and total tour duration (must complete within 180 minutes)."
    )
    
    # ---- Time Window Validation Agent ---- #
    TW_Agent = Agent(
        name="Time Window Validation Agent",
        backstory="You verify that all location visits occur within their specified time windows.",
        task_description="Validate that the planned route respects all time window constraints for each location.",
        task_expected_output="Validated schedule confirming all visits occur within time windows: library [9:00-12:00], cafeteria [11:00-14:00], gym [15:00-18:00]."
    )
    
    # ---- Tour Execution Agent ---- #
    TE_Agent = Agent(
        name="Tour Execution Agent",
        backstory="You execute the campus tour following the planned route and schedule.",
        task_description="Execute the validated tour plan, ensuring timely arrival at each location.",
        task_expected_output="Final tour execution plan with confirmed arrival times and route completion status."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p1_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    LS_Agent >> RP_Agent >> TW_Agent >> TE_Agent >> Writer_agent


In [ ]:
crew.run()
